In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
from dataloader_places import PlacesDataset
from torch.utils.data import DataLoader
# from domino._embed import embed
import clip
import torch
from tqdm import tqdm

idx_to_classname ={0: 'bathroom', 1: 'bedroom', 2: 'childs_room',
                   3: 'classroom', 4: 'dressing_room', 5: 'living_room',
                   6: 'studio', 7: "swimming_pool"}

data_path = "../adversarial-sets/data/Places8_paths_and_labels_complete_train.npy"
dataset = PlacesDataset(data_path)
batch_size = 64

dataloader = DataLoader(dataset, shuffle=True, num_workers=6, batch_size=batch_size)
print(f"\nDataloader: {batch_size} batch size | {len(dataloader)} batches | {len(dataloader.dataset)} images")

print(clip.available_models())
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)
model

In [ ]:
preprocess

In [ ]:
image_features = []
text_features = []
for inputs, labels, _ in tqdm(dataloader):

    text_inputs = torch.cat([clip.tokenize(
        f"a photo of a {idx_to_classname[label.item()]}", truncate=True) for label in labels]).to(device)
    # Calculate features
    with torch.no_grad():
        image_features.append(model.encode_image(inputs.to(device)))
        text_features.append(model.encode_text(text_inputs))


In [1]:
import numpy as np

image_features = np.asarray(image_features)
text_features = np.asarray(text_features)
np.save("places8_image_features_clip.npy", image_features)
np.save("places8_text_features_clip.npy", text_features)